In [92]:
import cv2 as cv
import numpy as np
import time

pic_name=".md-pictures/README/image-20230314094817514.png"


def show(img,img_name='image',filename="",call_back=None):
    cv.namedWindow(img_name)
    if not call_back is None:
        cv.setMouseCallback(img_name,call_back)
    while True:
        cv.imshow(img_name, img)
        k = cv.waitKey(20) # 等待输入
        if k & 0xFF == 27:  # 键盘输入esc退出
            break
        elif k == ord('s'):  # 键盘输入s保存
            cv.imwrite(filename, img)
            break
    cv.destroyAllWindows()
def keep(img,name):
    cv.imwrite(name, img)

def draw_point(img,pos,color=(0, 0, 255),size=2):
    cv.circle(img, pos,size, color,-1)
def sleep(t):
    time.sleep(t)
img=cv.imread(pic_name)
# 大渡口,江北termios

In [57]:
# 点击图像,并通过鼠标确定像素点颜色
# def check_by_mouse(pth):
#     def _check(event,x,y,flags,param):
#         if event == cv.EVENT_FLAG_LBUTTON:
#             temp=cv.imread(pth)
#             cv.namedWindow('image')
#             cv.setMouseCallback('image',_check)
#             print(x,y)
#             print(temp[y,x])
#             draw_point(temp,pos=(x,y))
#             cv.imshow('image',temp)
#             sleep(0.2)
#     temp=cv.imread(pth)
#     cv.namedWindow('image')
#     cv.setMouseCallback('image',_check)
#     cv.imshow('image',temp)
#
#     if cv.waitKey(0) & 0xFF == 27:
#         cv.destroyAllWindows()
#
# check_by_mouse(pic_name)

453 456
[250 250 250]
342 327
[250 228 202]
175 225
[250 228 202]
368 244
[242 220 194]
426 241
[242 220 194]
484 373
[242 220 194]


In [74]:
# # 分割蓝色部分
# def set_color(img,pos,_color):
#     if type(_color)!=tuple:
#         _color=(_color,_color,_color)
#     for i in range(3):
#         img.itemset((pos[0],pos[1],i),_color[i])
# def split_blue(img):
#     h,w,c=img.shape
#     for i in range(0,h):
#         for j in range(0,w):
#             a=img.item(i,j,0)
#             b=img.item(i,j,1)
#             c=img.item(i,j,2)
#             if a > b > c:
#                 set_color(img,(i,j),0)
#             else:
#                 set_color(img,(i,j),255)
#     return img
#
# down_flag=False
# color=(0,0,0)
#
# def fill(event,x,y,flags,param):
#     global down_flag,color
#     if event==cv.EVENT_RBUTTONDOWN:
#        if color==(0,0,0):
#            color=(255,255,255)
#        else:
#            color=(0,0,0)
#     elif (not down_flag) and event==cv.EVENT_LBUTTONDOWN:
#         down_flag=True
#     elif down_flag and event==cv.EVENT_LBUTTONUP:
#         down_flag=False
#     if down_flag:
#         draw_point(img_split_fill,(x,y),color,10)
#
#
# img_split_fill=split_blue(cv.imread(pic_name))
# show(img_split_fill,call_back=fill) # img始终保留分割并涂色后的图片


In [91]:
# #图像角点检测算法
# def goodFeaturesToTrack(img,MaxCorner,QualityLevel,MinDistance):
#     gray=cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#     gray = np.float32(gray)
#     res=cv.goodFeaturesToTrack(gray,MaxCorner,QualityLevel,MinDistance)
#     res=np.int0(res)
#     temp=img.copy()
#     points=set()
#     for i in res:
#         x,y=i.ravel()
#         draw_point(temp,(x,y),size=3)
#         points.add((x,y))
#     return temp,points
#
# MaxCorner = 50
# QualityLevel = 0.1
# MinDistance = 10
# img_points_check,points=goodFeaturesToTrack(img_split_fill,MaxCorner,QualityLevel,MinDistance)
#
# show(img_points_check)


In [94]:
# 点编辑器
## 写一个管理points的回调函数
## 维护一个points的有序数组
## remove时,每拖动一次鼠标,就检测是否触及到了红点并移除红点
## add时,直接把红点添加进set即可
## 每成功添加/删除一个红点,就要根据img和points集合重新绘图,并重新

down_flag=False
mode=0 # mode为0时代表删除点,mode为1时代表去除点
distance=5 #画笔删除时的宽度
points=[]
def dis(pos1,pos2):
    return ((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2)**0.5
def re_draw(): #重新画图
    img_to_show=img.copy()
    for i in points:
        draw_point(img_to_show,i,size=3)
    cv.namedWindow('image')
    cv.setMouseCallback('image',points_callback)
    cv.imshow('image',img_to_show)

def points_callback(event,x,y,flags,param):
    global down_flag,mode,points
    if event==cv.EVENT_RBUTTONDOWN: # 右键切换模式
       mode=(mode+1)%2
    elif (not down_flag) and event==cv.EVENT_LBUTTONDOWN:
        down_flag=True
    elif down_flag and event==cv.EVENT_LBUTTONUP:
        down_flag=False

    if down_flag and mode==0: # 删除模式
        size=len(points)
        points=[i for i in points if dis(i,(x,y))>=distance]
        if size != len(points): # 删除成功,重新绘制图
            re_draw()
    if event==cv.EVENT_LBUTTONDOWN and mode==1: #增加模式
        points.append((x,y))
        re_draw()

re_draw()
if cv.waitKey(0) & 0xFF == 27:
    cv.destroyAllWindows()

In [96]:
# 坐标计算
### 因为程序只知道角点的相对坐标,不知道绝对坐标
### 为了让程序正常输出绝对坐标,需要将图像内最低点的坐标告知程序
### 在计算坐标前首先要校准比例,校准比例可以通过最低点a,最高点b的绝对以及相对坐标来推算
### 比例 rate= (a.abs_y-b.abs_y)/(a.rel_y-b.rel_y)
### 其中abs为绝对坐标是高德地图给的,rel为绝对坐标是程序 points集合给的
### 为了方便使用,请不要调整地图比例,校准只需要第一次运行时才做
rate_x=1
rate_y=1
def calibration(): # 校准
    high=False
    for i in points:
        if not high or high[1]>=i[1]:
            high=i
    low=False
    for i in points:
        if not low or low[1]<=i[1]:
            low=i
    abs_high_x,abs_high_y,abs_low_x,abs_low_y=map(float,input("输入最高点,最低点的横,纵坐标,空格分割:").split(" "))
    global rate_y,rate_x
    rate_y=(abs_high_y-abs_low_y)/(low[1]-high[1]) #high实际上是y最小的点
    rate_x=(abs_high_x-abs_low_x)/(high[0]-low[0])
    print(rate_y,rate_x)
def coordinate_trans(rel_pos,rel_a,abs_a):
    res=((rel_pos[0]-rel_a[0])*rate_x+abs_a[0],(rel_a[1]-rel_pos[1])*rate_y+abs_a[1])
    return res
calibration()
# 106.550248 29.564839 106.551519 29.562525
# 29.562495
rel_low=False # 最低点的相对坐标
for i in points:
    if not rel_low or rel_low[1]<=i[1]:
        rel_low=i
abs_a_x,abs_a_y=(map(float,input("输入最低点横,纵坐标,逗号分割").split(",")))
abs_a=(abs_a_x,abs_a_y)
abs_points=[coordinate_trans(i,rel_low,abs_a) for i in points ]
print(abs_points)

5.247165532876129e-06 6.081339712931515e-06
[(106.54924966028707, 29.56345399546485), (106.5492557416268, 29.563978712018137), (106.5490733014354, 29.564408979591835), (106.550247, 29.564833999999998), (106.55036862679425, 29.564776281179135), (106.55229033014355, 29.562782358276642), (106.5517551722488, 29.56256197732426), (106.551518, 29.56252), (106.55083080861245, 29.563181142857143), (106.55044768421052, 29.5629030430839), (106.55011929186603, 29.563202131519272), (106.54975441148325, 29.563265097505667)]


In [67]:
print(5.235294117643378e-06/6.200000000013105e-06)

0.8444022770374696
